In [2]:
import torch
import sklearn
from sklearn import preprocessing
import pandas as pd
import geopandas as gpd
import rasterio
import os
import pandas as pd
import numpy as np
from torchvision.io import read_image

from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models

import torch.nn as nn


/home/jan/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
controls = pd.read_csv('data/ILUC_controls_labels.csv')
campaign = pd.read_csv('data/ILUC_campaign_labels.csv')



In [7]:
gdf = gpd.read_file('data/geowiki/ILUC.shp')

#I filter out locations so that I only have 3 or 4 answers per location
#---- I think with softmax I don't need to do it
#controls.set_index('sampleid').sum(axis=1)
#campaign = campaign.set_index('sampleid')
#campaign_filter = campaign.loc[campaign.sum(axis=1) > 2]
#campaign_filter = campaign_filter.loc[campaign_filter.sum(axis=1) < 5]

# I add geometry to the DataFrame with labels and write to disk.

indexed_geoms = gdf[['sampleid', 'geometry']].drop_duplicates().set_index('sampleid')
filtered_geoms = indexed_geoms.loc[campaign.index]
campaing_processed = pd.concat([campaign, filtered_geoms], axis=1)
#campaing_processed.to_csv("campaign_labels_processed.csv")'''

'''annotations_path = 'data/controls_labels_processed.csv'
labels = pd.read_csv(annotations_path)
paths = []
for i, row in labels.iterrows():
    filename = f'{row.sampleid}_2020_1.tif'
    paths.append(filename)

labels['filename'] = paths'''


"annotations_path = 'data/controls_labels_processed.csv'\nlabels = pd.read_csv(annotations_path)\npaths = []\nfor i, row in labels.iterrows():\n    filename = f'{row.sampleid}_2020_1.tif'\n    paths.append(filename)\n\nlabels['filename'] = paths"

In [30]:
gdf = gpd.read_file('data/geowiki/ILUC.shp')

#I filter out locations so that I only have 3 or 4 answers per location
#---- I think with softmax I don't need to do it
#controls.set_index('sampleid').sum(axis=1)
#campaign = campaign.set_index('sampleid')
#campaign_filter = campaign.loc[campaign.sum(axis=1) > 2]
#campaign_filter = campaign_filter.loc[campaign_filter.sum(axis=1) < 5]

# I add geometry to the DataFrame with labels and write to disk.

indexed_geoms = gdf[['sampleid', 'geometry']].drop_duplicates().set_index('sampleid')
filtered_geoms = indexed_geoms.loc[campaign_filter.index]
campaing_processed = pd.concat([campaign_filter, filtered_geoms], axis=1)
#campaing_processed.to_csv("campaign_labels_processed.csv")'''

'''annotations_path = 'data/controls_labels_processed.csv'
labels = pd.read_csv(annotations_path)
paths = []
for i, row in labels.iterrows():
    filename = f'{row.sampleid}_2020_1.tif'
    paths.append(filename)

labels['filename'] = paths'''


,Subsistence agriculture,Managed forest/forestry,Pasture,Roads/trails/buildings,Other natural disturbances/No tree-loss driver,Commercial agriculture,Wildfire (disturbance),Commercial oil palm or other palm plantations,Mining and crude oil extraction
sampleid,,,,,,,,,
1588043,1,2,0,0,0,0,0,0,0
1588045,1,0,2,0,0,0,0,0,0
1588046,2,0,2,0,0,0,0,0,0
1588047,3,0,0,0,0,0,0,0,0
1588048,1,0,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1743034,3,0,0,0,0,0,0,0,0
1743035,3,0,0,0,0,0,0,0,0
1743036,0,0,0,0,0,0,0,3,0


In [217]:
image_transform = torch.nn.Sequential(
    transforms.CenterCrop(32),
    transforms.Normalize((8472.0, 9534.1, 9378.7, 17898.8), (313.3, 431.8,  644.0, 1335.2)),
)


In [343]:
class GeoWikiDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx]['filename'])
        with rasterio.open(img_path) as source:
            image = torch.Tensor(source.read())
        labels = self.img_labels.iloc[idx].drop(['geometry', 'filename', 'sampleid']).to_dict()
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            labels = self.target_transform(labels)
        return image, labels

In [325]:
#Remove rows from labels where the image does not exist (wasn't created because it contains missing values)

'''annotations_path = 'data/controls_labels_processed.csv'

files = pd.read_csv(annotations_path)
print(files.shape)
for index, row in files.iterrows():
    if not os.path.isfile(os.path.join('data/controls_L8_examples/', row.filename)):
        files = files.drop(index)

files.to_csv( 'data/controls_labels_processed.csv', index=False)'''

In [344]:
annotations_path = 'data/controls_labels_processed.csv'
images_path = 'data/controls_L8_examples/'


full_dataset = GeoWikiDataset(annotations_file=annotations_path, img_dir=images_path, transform=image_transform)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [346]:
def compute_weights(labels):
    #TODO: handle batches
    """Compute weights to be used in loss function.
    If x votes were cast for one answer,
    assign it x-times higher weight.
    Negative labels have base weight

    Args:
        labels (Tensor): labels from geowiki dataset

    Returns:
        Tensor: weights
    """
    labels[labels == 0] = 1
    weights = labels/labels.sum()

    return weights

In [347]:
net = Net()
optimizer = torch.optim.Adam(net.parameters())
loss_func = torch.nn.BCEWithLogitsLoss(reduction='none')

def criterion(output, target_dict, loss_func):
    labels = torch.stack([value for key, value in target_dict.items()]).T
    labels_mask = labels.clone()
    labels_mask[labels_mask > 0] = 1

    loss_array = loss_func(output, labels_mask.float())
    weights = compute_weights(labels)
    loss = (weights*loss_array).mean()

    return loss

In [348]:
nb_epochs, batch_size =  10, 64

In [349]:
for epoch in range(nb_epochs):
    epoch_loss = 0
    for inputs, targets in train_dataloader:
        output = net(inputs)
        
        loss = criterion(output, targets, loss_func=loss_func)
        epoch_loss += loss.item()
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
    print(epoch_loss)


0.8540202745934948
0.7726436441298574
0.7235303529305384
0.7252430428052321
0.7313359236577526
0.7573005895828828
0.6854573612799868
0.6853354043560103
0.6795522067695856
0.720067159505561


In [8]:
input = torch.zeros((1,3,32,32))

In [39]:
net(input).shape

torch.Size([1, 1000])

In [36]:
net.layer4[0].conv1.stride = (1,1)
net.layer4[0].downsample[0].stride = (1,1)

In [37]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  